In [1]:
import pandas as pd
import glob as gb
import os

files = gb.glob('./data/Ward_Age/*')
dataname = '町丁目名年齢年代別人口分布'

d=os.getcwd()
prefix = d + '/../data/Ward_Age/'

bin_band =  list(range(0,131,5))


# c
#g_code = pd.read_csv(prefix+'東京都人口世帯数_estat.txt', encoding='SHIFT-JIS', dtype={'KEY_CODE':str})
#g_code[g_code['HYOSYO']>3]
#g_code=g_code[['KEY_CODE','NAME','CITYNAME']]

c_code = pd.read_csv(prefix + '13_2020.csv', encoding='utf-8', dtype={'大字町丁目コード':str})[['大字町丁目名','大字町丁目コード']]

In [2]:
c_code

,大字町丁目名,大字町丁目コード
0,内幸町２丁目,131010001002
1,内幸町１丁目,131010001001
2,霞が関３丁目,131010002003
3,北の丸公園,131010003000
4,富士見１丁目,131010004001
...,...,...
5359,母島,134210003000
5360,父島,134210004000
5361,聟島,134210005000
5362,南鳥島,134210006000


## 大田区

In [3]:
ohta_file = prefix + '大田区2022年1月_rev.xlsx'

ohta_data = pd.read_excel(ohta_file).set_index('年齢')

ohta_finedata = pd.DataFrame(ohta_data.stack()).reset_index().set_axis(['年齢','町丁目名','人口'],axis=1)

ohta_finedata = ohta_finedata[['町丁目名','年齢','人口']]

ohta_finedata['町丁目名'] = ohta_finedata['町丁目名'].str.replace('　','')

#ohta_finedata.to_excel(prefix + dataname + '（大田）.xlsx', index=False)

In [4]:
ew=pd.cut(ohta_finedata['年齢'], bin_band, right=False)
#pd.DataFrame(pd.DataFrame(ew).value_counts()).sort_values('年齢')

In [5]:
pd.merge(ohta_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名', how='left')

,町丁目名,年齢,人口,大字町丁目名,大字町丁目コード
0,大森東１丁目,0,24,大森東１丁目,131110039001
1,大森東２丁目,0,34,大森東２丁目,131110039002
2,大森東３丁目,0,30,大森東３丁目,131110039003
3,大森東４丁目,0,35,大森東４丁目,131110039004
4,大森東５丁目,0,27,大森東５丁目,131110039005
...,...,...,...,...,...
29031,新蒲田２丁目,121,0,新蒲田２丁目,131110034002
29032,新蒲田３丁目,121,0,新蒲田３丁目,131110034003
29033,ふるさとの浜辺,121,0,NaN,NaN
29034,令和島１丁目,121,0,令和島１丁目,131110062001


### 品川

In [6]:
male=list(range(0,241,2))
female=list(range(1,241,2))

shina_file = prefix + '品川区2023年1月_rev.xls'
shina_data = pd.read_excel(shina_file, dtype={'行政コード':int})

shina_m = shina_data.loc[:,['行政コード','丁目名']+male].set_index(['行政コード','丁目名']).stack()
shina_f = shina_data.loc[:,['行政コード','丁目名']+female].set_index(['行政コード','丁目名']).stack()

shina_m_fine= pd.DataFrame(shina_m).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_m_fine['年齢']=shina_m_fine['age']//2

shina_f_fine= pd.DataFrame(shina_f).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_f_fine['年齢']=(shina_f_fine['age'])//2

scols=['行政コード','丁目名','年齢','人口']
mer=['行政コード','丁目名','年齢']

shina_m_fine=shina_m_fine.drop(columns=['age'])[scols]
shina_f_fine=shina_f_fine.drop(columns=['age'])[scols]


shina_finedata = pd.merge(shina_m_fine, shina_f_fine, on=mer)

#for t,i in shina_finedata.iterrows():
#    shina_finedata.loc[i,'丁目名'] = shina_finedata.loc[i, '丁目名'].replace('\u3000','')
#    print(shina_finedata.loc[i, '丁目名'])

shina_finedata['人口'] = shina_finedata['人口_x']+shina_finedata['人口_y']
shina_finedata = shina_finedata.drop(columns=['人口_x','人口_y']).reset_index(drop=True)
shina_finedata = shina_finedata.rename(columns={'丁目名':'町丁目名'})
#shina_finedata.to_excel(prefix + dataname + '（品川）.xlsx', index=False)

In [7]:
c_code[c_code['大字町丁目名'].str.contains('品川')]

,大字町丁目名,大字町丁目コード
923,東品川４丁目,131090003004
925,東品川３丁目,131090003003
926,東品川２丁目,131090003002
928,東品川１丁目,131090003001
934,東品川５丁目,131090003005
979,南品川５丁目,131090017005
980,南品川２丁目,131090017002
981,南品川３丁目,131090017003
982,南品川４丁目,131090017004
983,南品川１丁目,131090017001


In [8]:
shina_finedata.loc[4,'町丁目名']

'北品川１丁目'

In [9]:
pd.merge(shina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名', how='left')

,行政コード,町丁目名,年齢,人口,大字町丁目名,大字町丁目コード
0,1101,北品川１丁目,0,20,北品川１丁目,131090019001
1,1101,北品川１丁目,1,17,北品川１丁目,131090019001
2,1101,北品川１丁目,2,21,北品川１丁目,131090019001
3,1101,北品川１丁目,3,18,北品川１丁目,131090019001
4,1101,北品川１丁目,4,14,北品川１丁目,131090019001
...,...,...,...,...,...,...
16195,6200,東八潮,115,0,東八潮,131090005000
16196,6200,東八潮,116,0,東八潮,131090005000
16197,6200,東八潮,117,0,東八潮,131090005000
16198,6200,東八潮,118,0,東八潮,131090005000


### 目黒

In [10]:
meg_file = prefix +'目黒区2022年12月_rev.xlsx'
meg_data = pd.read_excel(meg_file)
#meg_data

top = meg_data[meg_data['a']=='町丁別年齢別人口表'].reset_index()['index']
end = meg_data[meg_data['a']=='104以上'].reset_index()['index']
#print(list(top))
#print(list(end))

meg_tmp = pd.DataFrame()

#meg_data['町丁目名']=''

for k in top:
    chomei = meg_data.iloc[k+1,0].replace('　','')

    for j in list(range(k+4,k+129)):
        meg_data.loc[j,'町丁目名'] = chomei

    chodf = meg_data.iloc[(k+4):(k+129),:]
    
    chodf = chodf[~(chodf['a'].str.contains('～',na=False))]
    chodf = chodf[~(chodf['a'].str.contains('以上',na=False))]
    
    chodf = chodf.rename(columns={'a':'年齢', 'b':'人口'})
    chodf['年齢'] = chodf['年齢'].astype(int)
    
    meg_tmp = pd.concat([meg_tmp, chodf])

meg_finedata = meg_tmp[['町丁目名','年齢','人口']]

#meg_finedata.to_excel(prefix + dataname + '（目黒）.xlsx', index=False)

In [11]:
meg_finedata

,町丁目名,年齢,人口
5,駒場１丁目,0,30
6,駒場１丁目,1,16
7,駒場１丁目,2,26
8,駒場１丁目,3,19
9,駒場１丁目,4,32
...,...,...,...
12129,東が丘２丁目,99,3
12131,東が丘２丁目,100,0
12132,東が丘２丁目,101,1
12133,東が丘２丁目,102,0


In [12]:
pd.merge(meg_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名', how='left')

,町丁目名,年齢,人口,大字町丁目名,大字町丁目コード
0,駒場１丁目,0,30,駒場１丁目,131100027001
1,駒場１丁目,1,16,駒場１丁目,131100027001
2,駒場１丁目,2,26,駒場１丁目,131100027001
3,駒場１丁目,3,19,駒場１丁目,131100027001
4,駒場１丁目,4,32,駒場１丁目,131100027001
...,...,...,...,...,...
10811,東が丘２丁目,99,3,東が丘２丁目,131100017002
10812,東が丘２丁目,100,0,東が丘２丁目,131100017002
10813,東が丘２丁目,101,1,東が丘２丁目,131100017002
10814,東が丘２丁目,102,0,東が丘２丁目,131100017002


### 港区

In [13]:
import tabula

mina_file = prefix + '港区2022年4月.pdf'

mina_data = tabula.read_pdf(mina_file,pages="1-117",stream=True)

In [14]:
textfile = open(prefix+'港区町丁目名.txt','r')
output_text = textfile.read()
textfile.close()

import re

pattern = '(.+丁目)|(麻布.+町)|(麻布狸穴町)$'

minacho = re.findall(pattern, output_text)

from itertools import chain
def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

#flatten(list(map(lambda x: list(x), minacho)))

chomeilist = list(map((lambda y: ''.join(y)), list(map(lambda x: list(x), minacho))))

In [15]:
mina_finedata = pd.DataFrame()

for i in range(0,116):
    if i != 39:
        k = mina_data[i].iloc[:21,0:3].set_axis(['世代','a','b'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k = k.fillna(0)
        k = k.astype({'a':str, 'b':str})
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)
        k['b'] = k['b'].str.replace(',','')
        k['b'] = k['b'].str.replace(r'0\.0','0', regex=True)
    else:
        k = mina_data[i].iloc[:,0:2].set_axis(['世代','a'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)        
        k['b'] = '0'
        
    mina_finedata = pd.concat([mina_finedata, k])

mina_finedata = mina_finedata.astype({'a':int, 'b':int}).reset_index(drop=True) 

mina_finedata['人口'] = mina_finedata['a'] + mina_finedata['b']
#

for r in mina_finedata.itertuples():
    if not(((r.a == 0) and (r.b==r.人口)) or ((r.b==0) and (r.a==r.人口))):
        print(r.町丁目名, ' ', r.a, r.b)


mina_finedata = mina_finedata.drop(columns=['a','b'])[['町丁目名','世代','人口']]
#mina_finedata.to_excel(prefix + dataname + '（港）.xlsx', index=False)

In [16]:
c_code[c_code['大字町丁目名'].str.contains('赤坂')]

,大字町丁目名,大字町丁目コード
226,赤坂１丁目,131030008001
227,赤坂２丁目,131030008002
228,赤坂６丁目,131030008006
229,赤坂７丁目,131030008007
230,赤坂８丁目,131030008008
313,赤坂９丁目,131030008009
323,赤坂５丁目,131030008005
324,赤坂３丁目,131030008003
325,元赤坂１丁目,131030030001
326,赤坂４丁目,131030008004


In [17]:
pd.merge(mina_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名', how='left')

,町丁目名,世代,人口,大字町丁目名,大字町丁目コード
0,赤坂１丁目,0~4歳,35,赤坂１丁目,131030008001
1,赤坂１丁目,5~9歳,37,赤坂１丁目,131030008001
2,赤坂１丁目,10~14歳,25,赤坂１丁目,131030008001
3,赤坂１丁目,15~19歳,22,赤坂１丁目,131030008001
4,赤坂１丁目,20~24歳,20,赤坂１丁目,131030008001
...,...,...,...,...,...
2519,台場１丁目,80~84歳,89,台場１丁目,131030010001
2520,台場１丁目,85~89歳,50,台場１丁目,131030010001
2521,台場１丁目,90~94歳,19,台場１丁目,131030010001
2522,台場１丁目,95~99歳,7,台場１丁目,131030010001


### 渋谷

In [23]:
shib_data = pd.read_excel(prefix + '渋谷区2022年1月.xls', header=1)

shib_finedata = shib_data[['町丁名','年齢','人口']].rename(columns={'町丁名':'町丁目名'})

#shib_finedata.to_excel(prefix + dataname + '（渋谷）.xlsx', index=False)

In [19]:
shib_finedata

,町丁名,年齢,人口
0,恵比寿１丁目,2,1
1,恵比寿１丁目,7,1
2,恵比寿１丁目,14,1
3,恵比寿１丁目,20,1
4,恵比寿１丁目,21,1
...,...,...,...
11443,神宮前６丁目,98,1
11444,神宮前６丁目,99,2
11445,神宮前６丁目,102,1
11446,神宮前６丁目,103,1


In [25]:
pd.merge(shib_finedata, c_code, left_on='町丁目名', right_on='大字町丁目名', how='left')

,町丁目名,年齢,人口,大字町丁目名,大字町丁目コード
0,恵比寿１丁目,2,1,恵比寿１丁目,131130010001
1,恵比寿１丁目,7,1,恵比寿１丁目,131130010001
2,恵比寿１丁目,14,1,恵比寿１丁目,131130010001
3,恵比寿１丁目,20,1,恵比寿１丁目,131130010001
4,恵比寿１丁目,21,1,恵比寿１丁目,131130010001
...,...,...,...,...,...
17575,神宮前６丁目,98,1,神宮前６丁目,131130001006
17576,神宮前６丁目,99,2,神宮前６丁目,131130001006
17577,神宮前６丁目,102,1,神宮前６丁目,131130001006
17578,神宮前６丁目,103,1,神宮前６丁目,131130001006


### 台東